# 自定义模块和包

Python 系统除了提供自带的标准库外，自己还会安装大量的外部库。在 Python 编程中，仍然还会实现自己的模块与包。本节会介绍如何实现自定义模块与包，不过在动手之前，不要忘了这句话：
> 不要重复造轮子(Stop Trying to Reinvent the Wheel)

## 模块(module)

在 Python 中一个模块就是一个文件，编写Python代码并用“.py”后缀名进行保存，也就编写了一个 Python 模块。

模块名字应该尽量用简短全小写的名字，还是一句老话，做到望文生义不容易。

模块编写通常采用“高内聚低耦合”的思想原则：
- 内聚，描述模块内各个元素彼此结合的紧密程度，高内聚就是一个模块内各个元素彼此结合的紧密程度高。
- 耦合，不同模块之间互连程度的度量，低耦合是指模块尽可能独立完成某特定功能。
也就是说，一个模块尽量做到代码相关性强、功能独立。

### 创建模块

下面来创建自己的一个模块，假定模块包括一些常量、函数和类。

In [1]:
%%writefile simplemodule.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""简单模块

- 函数、类、变量
"""
__author__ = 'Wang Weihua'

hello = ['Hi', '您好']

def sayhello(name):
    """Say hello to a person"""
    print('Hi, {}'.format(name))

    
class SimpleClass(object):
    pass

Writing simplemodule.py


### 模块导入

可以使用 `import` 命令导入模块:

In [ ]:
import simplemodule

由于要导入的模块名字比较长或者其它原因，可以使用如下导入语法：
```
import module_name as qualname
```
创建新的模块对象：

In [ ]:
import simplemodule as sm

模块对象导入后，就可以使用`.`操作符来访问模块中的函数、类或对象：

In [ ]:
print(sm.hello)

In [ ]:
sm.sayhello('Peter')

In [ ]:
obj1 = sm.SimpleClass()
print(obj1)

### 自省

使用内置函数`type()`查看模块对象的类型：

In [ ]:
print(type(simplemodule), type(sm))

模块对象是可变对象，也就是说，可以增加、更改、删除模块中的属性：

In [ ]:
sm.hello = 'hello world'
sm.newdata = 'New Data'
print(sm.hello, sm.newdata)

模块对象一般由如下魔术属性：
- `__author__`, 作者
- `__doc__`, 文档字符串
- `__file__`, 模块文件路径
- `__name__`, 模块名字

In [ ]:
print(sm.__author__)
print(sm.__doc__)
print(sm.__file__)
print(sm.__name__)

使用内置函数 `help()` 函数或者 IPython 自省功能可以查看模块的帮助：

In [ ]:
help(sm)

### 重复导入

当导入一个模块时，Python解释器会执行模块文件中的语句，并创建一个模块对象。在不同地方，经常会导入同一个模块。但要注意的是，当多次导入同一模块时，解释器并没有运行两次，也就是模块不会重复导入，这样可以节省内存和计算资源。

![模块对象](../images/module_objects.png)

在开发调试时，经常需要重新导入，可以使用 `importlib.reload` 实现重复导入：

In [ ]:
import importlib

importlib.reload(sm)

## 程序与模块

一个 Python 代码文件可以是程序文件，也可以是模块文件。当用 Python 运行文件时就是主程序，在程序文件使用 `import` 命令导入时，就是模块。

一个 Python 代码文件有一个特殊属性 `__name__`，在这两种场景下其值并不同：
- 使用 Python 运行文件时，也就是说为主程序时其值被设置为`__main__`。
- 使用 `import` 命令导入模块时， 其值为文件名。

基于此特性，编写新的代码文件：

In [ ]:
%%writefile simplemodule2.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""My Simple Module

This exports:
  - all simple functions and class.
  - sm.sayhello is either posixpath or ntpath
  - sm.SimpleClass is either 'posix' or 'nt'

Programs that import and use 'sm' name 
"""
__author__ = 'Wu Yang'

hello = ['Hi', '您好']

def sayhello(name):
    """Say hello to a person"""
    print('Hi, {}'.format(name))

    
class SimpleClass():
    pass

print(__name__)
if __name__ == '__main__':
    for x in hello:
        print(x)
        
    sayhello('老王')

使用 `import` 语句导入模块，则模块的特殊属性`__name__`为模块名字：

In [ ]:
import simplemodule2 as sm2

print(sm2.__name__)

在命令行下，使用 Python 解释器来运行程序，则其特殊属性`__name__`为`__main__`：

In [ ]:
%run simplemodule2.py

基于这个特性，编写 Python 文件时常常用如下语句：
```
if __name__ == '__main__':
    suits
```

当文件作为模块时，不执行该语句块；当文件作为程序运行时则执行该语句块。

## 包（package）

在 Python 中，一个模块就是一个文件，一个软件包就是一个目录，使用多层目录，软件包可以嵌套。Python 包是模块的集合，是更高一级的封装。

### 创建包

一个目录就是软件包，不过区别在于，目录必须包含一个`__init__.py`文件。包是模块的集合，通常包含多个多个模块或子包。从理论上说，一个最简单的包允许只是一个目录（只有`__init__.py`文件），不过如果是这样的话，为啥不用模块呢？

下面来创建一个最简单的包，只包含`__init__.py`。也就是创建一个目录，然后在目录下创建一个`__init__.py`空文件：

In [ ]:
%mkdir simpkg
!touch simpkg/__init__.py

In [ ]:
!tree simpkg

### 导入包

使用`import`来导入软件包。由于一个软件包可能会包含子包以及模块，可以使用如下语法：
```
# 导入包
import dir1
# 导入子包中的模块
import dir1.dir2.module
# 导入子包中的模块
from dir1.dir2 import module
```

In [ ]:
import simpkg
import simpkg as sm

### 自省

使用内置函数`type()`查看对象的类型

In [ ]:
print(type(simpkg), type(sm))

这说明，包也是模块的实例对象。查看软件包的特殊属性：

In [ ]:
simpkg.__file__

### 包含模块与子包

下面在`__init__.py`添加一些内容，再添加一个新的模块，以及一个子包。

In [ ]:
%%writefile simpkg/__init__.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""My Simple Package

simple package include a module and a subpackage
"""
__author__ = 'Wang Weihua <whwang@isciencehub.com>'
__status__  = "production"
__version__ = "0.0.1.1"
__date__    = "24 July 2018"

hello = ['Hi', '您好']

def sayhello(name):
    """Say hello to a person"""
    print('Hi, {}'.format(name))

    
class SimpleClass():
    pass

在包中添加一个模块，也就是在目录中添加一个文件：

In [ ]:
%%writefile simpkg/multilang.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
def loveyou():
    """say love in multi language"""
    print('我爱你')    
    print('I Love You')
    print('私は爱する')             # 日本：
    print('나는 너를 사랑한다')     # 韩国
    print('Я люблю вас')   # 俄罗斯
    print("Je t'aime")              # 法国
    print('Ich liebe dich')         # 德国
    print('∑ας αγαπώ')       # 希腊
    print('Te amo')                 # 西班牙
    print('Ik houd van u')          # 荷兰

在包中添加一个子包，也就是创建一个子目录和`__init__.py`文件：

In [ ]:
%mkdir -p simpkg/fibo
!touch simpkg/fibo/__init__.py

In [ ]:
%%writefile simpkg/fibo/fibo.py
"""Fibonacci numbers module
"""

def fibo_gen(n):
    """Return Fibonacci series uup to n"""
    result = []
    a, b = 0, 1
    while b < n:
        result.append(b)
        a, b = b, a+b
    
    return result

def fibo_print(n):
    """Print a Fibonacci series"""
    print(fibo_gen(n))

使用`tree`命令列出包的内容

In [ ]:
!tree simpkg

### 导入包与调用

使用`import`导入需要的软件包：

In [ ]:
importlib.reload(sm)
import simpkg.multilang as mlang
from simpkg.fibo import fibo

使用`help()`查看软件包的帮助：

In [ ]:
# help(sm)

调用包中的函数：

In [ ]:
sm.sayhello('老王')
fibo.fibo_print(5)
sm.multilang.loveyou()

## 模块路径

在 Python 编程中，会用到：
- 标准库
- 外部库
- 自定义库

模块和包有很多，难免会出现重名现象。当使用 `import` 来加载模块会使用哪一个呢？ Python 在导入模块时依赖一个路径列表（`sys.path`），会根据这个列表中的路径去搜索要加载的模块：

In [ ]:
import sys

print(sys.path)

`sys.path` 是一个列表，可以进行添加或删除。在 `sys.path` 中目录顺序很重要，Python 通过遍历这个列表来寻找请求的模块。总体而言，模块的搜索顺序如下：
1. Python 安装目录
2. 外部库 `site-packages` 目录。
3. 当前目录

在Linux系统下，还可以通过修改环境变量`PYTHONPATH`，改变搜索路径。

## 错误与异常

在导入模块时，经常遇到的错误是`ModuleNotFoundError`异常，`ImportError`异常的子类。当导入模块没有安装或者模块名字拼写错误时，会抛出该异常：

In [ ]:
import nomodule

在一些情况下下，可以选用多个类似功能的模块，使用`try..except`方法来实现：
```
try:
    import A
except ImportError:
    import B
```    